# Lightweight Fine-Tuning Project

TODO: In this cell, describe your choices for each of the following

* PEFT technique: 
* Model: 
* Evaluation approach: 
* Fine-tuning dataset: 

#### PEFT
PEFT is used to leverage large models that have previously been trained. It achieves this by updating only a small set of parameters, thus reducing computational resources. The small set of parameters could be the last layer in a multilevel neural network. 

A version of PEFT, Low Rank Adaptation or Loram, will be used in this example. This technique creates a 'delta' layer using fewer parameters than the original layer. 
#### Model
The model chosen is one of the transformer models - AutoModelForSequenceClassification - which is one of the text sentiment models. It contains only two features, text and label.  

## Loading and Evaluating a Foundation Model

TODO: In the cells below, load your chosen pre-trained Hugging Face model and evaluate its performance prior to fine-tuning. This step includes loading an appropriate tokenizer and dataset.

#### Load Pretrained Model

In [1]:
import transformers
print(transformers.__version__)
!python --version
import numpy as np
import os
import torch
from transformers import DataCollatorWithPadding
from transformers import AutoModelForSequenceClassification

4.36.0
Python 3.10.11


In [2]:
from datasets import load_dataset,get_dataset_split_names,load_dataset_builder
dataset = load_dataset("imdb")
dataset_builder = load_dataset_builder("imdb")
#dataset_builder.info.description

In [3]:
splits = ["train","test"]
dataset = {split: dataset for split, dataset in zip(splits,load_dataset("imdb",split=splits))}

In [4]:
for split in splits:
    dataset[split] = dataset[split].shuffle(seed=42).select(range(500))

#### Dataset

### Model Name

In [5]:
model_name = "gpt2"
#model_name = "distilbert-base-uncased"

In [7]:
from transformers import AutoTokenizer 
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [9]:
def preprocess_function(examples):
    return tokenizer(examples["text"],padding="max_length",truncation=True)

In [10]:
splits=["train", "test"]
tokenized_ds = {}

In [11]:
for split in splits:
    tokenized_ds[split] = dataset[split].map(
        lambda x: tokenizer(x["text"], truncation=True), batched=True)
    

### Load Model Prior to Fine Tuning

In [12]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return {"accuracy": (predictions == labels).mean()}

### Model Definition

In [14]:
from transformers import AutoModelForSequenceClassification, Trainer, TrainingArguments
model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels = 2,
    id2label={0: "positive", 1: "negative"},
    label2id={"positive":0, "negative":1})

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [15]:
model.config.pad_token_id = tokenizer.eos_token_id

In [16]:
for param in model.parameters():
    param.requires_grad = True

In [17]:
from transformers import Trainer, DataCollatorWithPadding
trainer = Trainer(
model=model,
args=TrainingArguments(
    output_dir="./data",
    learning_rate=2e-3,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=1,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    ),
    train_dataset=tokenized_ds["train"],
    eval_dataset=tokenized_ds["test"],
    tokenizer=tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer),

    compute_metrics=compute_metrics,
)
    
    

In [18]:
trainer.train()

You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


ValueError: Asking to pad but the tokenizer does not have a padding token. Please select a token to use as `pad_token` `(tokenizer.pad_token = tokenizer.eos_token e.g.)` or add a new pad token via `tokenizer.add_special_tokens({'pad_token': '[PAD]'})`.

In [ ]:
import pandas as pd

items_for_manual_review = tokenized_ds["train"].select(
[0,1,22])

results = trainer.predict(items_for_manual_review)
df = pd.DataFrame(
{
    "text":[item["text"] for item in items_for_manual_review],
    "predictions": results.predictions.argmax(axis=1),
    "labels":results.label_ids,
    
})
pd.set_option("display.max_colwidth",None)
df

In [ ]:
trainer.evaluate()

### PEFT Model

In [ ]:
#from transformers import task_type
from peft import get_peft_config, get_peft_model, LoraConfig
#AutoPeftModelForCausalLM
import re
pattern = r'\((\w+)\): Linear'
linear_layers = re.findall(pattern, str(model.modules))
target_modules = list(set(linear_layers))

In [ ]:
print(linear_layers)

In [ ]:
peft_config = LoraConfig(
    r=8,
    lora_alpha=16,
    #target_modules=target_modules,
    target_modules =  target_modules,
    #lora_dropout=0.01,
    task_type="SEQ_2_SEQ_LM",
    #task_type = "CAUSAL_LM"
)

In [ ]:
#from peft import get_peft_model
peft_model = get_peft_model(model, peft_config)

In [ ]:
for param in peft_model.parameters():
    param.requires_grad = False

## Performing Parameter-Efficient Fine-Tuning

TODO: In the cells below, create a PEFT model from your loaded model, run a training loop, and save the PEFT model weights.

In [ ]:
trainer = Trainer(
model=peft_model,
args=TrainingArguments(
    output_dir="./data",
    learning_rate=2e-3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=5,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
   
    ),
    train_dataset=tokenized_ds["train"],
    eval_dataset=tokenized_ds["test"],
    #tokenizer=tokenizer,
    #data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
    compute_metrics=compute_metrics,
)



In [ ]:
trainer.train()

## Performing Inference with a PEFT Model

TODO: In the cells below, load the saved PEFT model weights and evaluate the performance of the trained PEFT model. Be sure to compare the results to the results from prior to fine-tuning.